## Метод Гауса

In [1]:
import numpy as np
import copy
import random
import sys

##### Введіть, будь ласка, матрицю A системи  порядково, використовуючи space як розділювач, і вектор b

In [22]:
# input the data from keyboard
error = True
while error:
    try:
        rows_number = int(input('Input the number of rows:'))
        columns_number = int(input('Input the number of columns:'))
        matrix_size = [rows_number, columns_number]
        print('\n' + 'The size of matrix is ' + str(rows_number) + ' x ' + str(columns_number) + '\n')
        
        matrix = []
        while (rows_number != 0):
            row = input('Input the row of the matrix as a separator use space:')
            row = row.split(' ')
            row = [float(x) for x in row]
            matrix.append(row)
            rows_number = rows_number - 1
        
        vector_b = []
        row = input('Input the vector b as a separator use space:')
        row = row.split(' ')
        vector_b = [float(x) for x in row]
        error = False
        
        print('\n' + 'The matrix A is:')
        print(np.array(matrix))
        print('\n' + 'The vector b is:')
        vector_b1 = np.array(vector_b)
        print(vector_b1.reshape(len(vector_b1), 1))
        
    except ValueError:
        error = True
        print("Incorrent format. Try again")
    

Input the number of rows:3
Input the number of columns:3

The size of matrix is 3 x 3

Input the row of the matrix as a separator use space:2 -7 1
Input the row of the matrix as a separator use space:3 -1 8
Input the row of the matrix as a separator use space:10 -2 3
Input the vector b as a separator use space:15 12 18

The matrix A is:
[[ 2. -7.  1.]
 [ 3. -1.  8.]
 [10. -2.  3.]]

The vector b is:
[[15.]
 [12.]
 [18.]]


#### Приклади з варіанту №4

In [3]:
A_1 = [[2, 2, -1, 1], [-3, 0, 3, 0], [-1, 3, 3, 2], [1, 0, 0, 4]]
b_1 = [3, -9, -7, 4]

A_2 = [[0, 3, -1], [9, 24, 1], [21, -1, -16]]
b_2 = [7, 20, 63]

A_3 = [[5, 0, -7, 0], [-1, 6, 0, 1], [2, -6, -4, -5], [-6, -6, 15, 7]]
b_3 = [-123, 60, -108, 159]

#### Допоміжні функції

In [4]:
def switch_rows(matrix, i, j): # switch the i row to j row
    k = matrix[i]
    matrix[i] = matrix[j]
    matrix[j] = k
    return matrix

def find_main_element(matrix, j): # return the main element and number of row where it is
    main_element = matrix[j][j]
    row_number = j
    for i in range(j+1, len(matrix)):
        if abs(main_element) < abs(matrix[i][j]):
            main_element = matrix[i][j]
            row_number = i
    return main_element, row_number

def count_determinant(triangular_matrix, permutations): 
    determinant = (-1) ** permutations
    for i in range(len(triangular_matrix)):
        determinant = determinant * triangular_matrix[i][i]
    return determinant 

def count_norm_matrix(matrix): # Октаєдрична норма
    max_sum = 0
    for j in range(len(matrix)):
        sum_column = 0
        for i in range(len(matrix)):
            sum_column += abs(matrix[i][j])
        if (max_sum < sum_column):
            max_sum = sum_column
    return max_sum

def move_up(under_triangular_matrix, vector_b):
    #error = False
    size =len(under_triangular_matrix)
    x = np.zeros(size)
    if (under_triangular_matrix[size - 1][size - 1] != 0): 
        x[size - 1] = vector_b[size - 1] / under_triangular_matrix[size - 1][size - 1]
        i = size - 2
        while(i >= 0):
            s = 0
            for j in range(i+1, size):
                s += under_triangular_matrix[i][j] * x[j]
            x[i] = (vector_b[i] - s) / under_triangular_matrix[i][i]
            i = i - 1
    #else:
    #    error = True
    return x

def move_down(down_triangular_matrix, vector_b):
    #error = False
    size =len(down_triangular_matrix)
    x = np.zeros(size)
    if (down_triangular_matrix[0][0] != 0):
        x[0] = vector_b[0] / down_triangular_matrix[0][0]
        i = 1
        while (i < size):
            s = 0
            for j in range(i):
                s += down_triangular_matrix[i][j] * x[j]
            x[i] = (vector_b[i] - s) / down_triangular_matrix[i][i]
            i = i + 1
    #else:
    #    error = True
    return x

In [5]:
# solution fo the system with Gauss` method
def Gauss(m_A, v_b, eps):
    
    matrix = copy.deepcopy(m_A)
    vector_b = copy.deepcopy(v_b)
    
    error = False
    permutations = 0
    for j in range(len(matrix) - 1):
        main_element, row_number  = find_main_element(matrix, j)
        #print(main_element)
        if (abs(main_element) < eps): 
            error = True
            break
        else:
            if (row_number != j):
                matrix = switch_rows(matrix, j, row_number)
                vector_b = switch_rows(vector_b, j, row_number)
                permutations += 1
            for i in range(j+1, len(matrix)):
                t = matrix[i][j] / main_element
                vector_b[i] = vector_b[i] - t * vector_b[j]
                #print(vector_b)
                matrix[i][j] = 0
                for k in range(j+1, len(matrix)):
                    matrix[i][k] = matrix[i][k] - t * matrix[j][k]
                #print(matrix) 
    if not error:
        x = np.zeros(len(matrix))
        if (matrix[len(matrix) - 1][len(matrix) - 1] != 0): 
            x[len(matrix) - 1] = vector_b[len(matrix) - 1] / matrix[len(matrix) - 1][len(matrix) - 1]
            i = len(matrix) - 2
            while(i >= 0):
                s = 0
                for j in range(i+1, len(matrix)):
                    s += matrix[i][j] * x[j]
                x[i] = (vector_b[i] - s) / matrix[i][i]
                i = i - 1
            determinant = count_determinant(matrix, permutations)
            return matrix, determinant, x, error, row_number
        else:
            error = True
 
    return matrix, 0, 0, error, row_number


In [6]:
# LU factorization

def factorizate_matrix(matrix):
    error = False
    L_matrix = np.zeros_like(matrix, dtype = float)
    U_matrix = np.zeros_like(matrix, dtype = float)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i == j):
                L_matrix[i][i] = 1
                                                       
            if (i <= j):
                sum_1 = 0
                for k in range(i):
                    sum_1 += L_matrix[i][k] * U_matrix[k][j]
                U_matrix[i][j] = matrix[i][j] - sum_1
            
            if (i > j):
                sum_2 = 0
                for n in range(j):
                    sum_2 += L_matrix[i][n] * U_matrix[n][j]
                if (U_matrix[j][j] != 0):    
                    L_matrix[i][j] = (1 / U_matrix[j][j]) * (matrix[i][j] - sum_2)
                else:
                    error = True
                    break

    return L_matrix, U_matrix, error

In [7]:
# reverse matrix based on LU factorisation

def find_reverse_matrix(L_matrix, U_matrix):
    reverse_matrix = np.zeros_like(L_matrix, dtype = float)
    
    for j in range(len(U_matrix)):
        one_vector = np.zeros(len(L_matrix))
        y = np.zeros(len(L_matrix))
        x = np.zeros(len(L_matrix))
        one_vector[j] = 1
        y = move_down(L_matrix, one_vector)
        x = move_up(U_matrix, y)
        
        for k in range(len(x)):
            reverse_matrix[k][j] = x[k]
        
    return reverse_matrix



#### Збурення вектора вільних членів

In [25]:
def perturbation(b, number_coordinates):
    permutation = np.random.permutation(range(len(b)))
    for k in range(number_coordinates):
        b[permutation[k]] += 0.01 * b[permutation[k]]
    return b

def compare_solutions(A, b):
    epsilon = sys.float_info.epsilon
    initial_A = copy.deepcopy(A)
    initial_b = copy.deepcopy(b)
    new_b = perturbation(b, number_coordinates=3)
    print('Initial data')
    print(np.array(initial_A))
    print(initial_b)
    print('The new vector b')
    print(new_b)
    
    matrix_A_1, det, x_1, error_Gauss, row = Gauss(initial_A, initial_b, epsilon)
    matrix_A_2, det, x_2, error_Gauss, row = Gauss(A, new_b, epsilon)
    print('The solution without perturbation')
    print(x_1)
    print('\n' + 'The solution with perturbation 1%')
    print(x_2)
    print('\n'+ 'Absolute difference:')
    print(abs(x_1 - x_2))

In [9]:
def print_answers(A, b):
    epsilon = sys.float_info.epsilon
    initial_A = copy.deepcopy(A)
    initial_b = copy.deepcopy(b)
    initial_b1 = np.array(initial_b)
    matrix_A, det, x, error_Gauss, row = Gauss(A, b, epsilon)
    L, U, error_LU = factorizate_matrix(initial_A)
    
    print('The initial matrix A and the vector b')
    print(np.array(initial_A)) 
    print(initial_b1.reshape(len(initial_b), 1))

    if error_Gauss:
        print('The determinant of the matrix A is 0')
        print('\n' + 'Zero main element occured in the row #' + str(row))
        print('\n' + 'Answer of libraries')
        print('The determinant of the matrix A: ')
        print(np.linalg.det(initial_A))
        print('\n' + 'The matrix can not be factorized in L and U')
        print('\n' + 'The reverse matrix does not exist')
    else:
        print('The determinant of the matrix A:')
        print(det)
        print('The solution of the system:')
        print(x)
    
        initial_A = np.array(initial_A)
        initial_b = np.array(initial_b)
        print('\n' + 'Answers of libraries')
        print('The determinant of the matrix A: ')
        print(np.linalg.det(initial_A))
        print('\n' + 'The solution of the system:' )
        print(np.linalg.solve(initial_A, initial_b))
    
        print('\n' + 'The error of solution:')
        print(initial_A.dot(x) - initial_b)
    
        if not error_LU:
            print('\n' + 'The matrix L')
            print(L)
            print('\n' + 'The matrix U')
            print(U)
            print('LU - A')
            print(L.dot(U) - initial_A)
            
            rev_A = find_reverse_matrix(L, U)
            print('\n' + 'The reverse matrix')
            print(rev_A)
            print('\n' + 'The product of the matrix and the reverse matrix')
            print(rev_A.dot(initial_A))
            
            cond_A = count_norm_matrix(initial_A) * count_norm_matrix(rev_A)
            print('\n' + 'The conditional number')
            print(cond_A)
        else:
            print('\n' + 'The matrix can not be factorized in L and U because one of minor is zero')
            #print('\n' + 'To find the reverse matrix th has been used')
            
            initial_A1 = copy.deepcopy(initial_A)
            rev_A = np.linalg.inv(initial_A)
            print('\n' + 'The reverse matrix')
            print(rev_A)
            print('\n' + 'The product of the matrix and the reverse matrix')
            print(rev_A.dot(initial_A1))
            
            cond_A = count_norm_matrix(initial_A1) * count_norm_matrix(rev_A)
            print('\n' + 'The conditional number')
            print(cond_A)
            
    
    

#### Результати для матриці 1 з варіанту №4

In [10]:
start_A_1 = copy.deepcopy(A_1)
start_b_1 = copy.deepcopy(b_1)
print_answers(A_1, b_1)

The initial matrix A and the vector b
[[ 2  2 -1  1]
 [-3  0  3  0]
 [-1  3  3  2]
 [ 1  0  0  4]]
[[ 3]
 [-9]
 [-7]
 [ 4]]
The determinant of the matrix A:
8.999999999999998
The solution of the system:
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

Answers of libraries
The determinant of the matrix A: 
8.999999999999998

The solution of the system:
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

The error of solution:
[ 0.0000000e+00  0.0000000e+00 -8.8817842e-16  0.0000000e+00]

The matrix L
[[ 1.          0.          0.          0.        ]
 [-1.5         1.          0.          0.        ]
 [-0.5         1.33333333  1.          0.        ]
 [ 0.5        -0.33333333  2.          1.        ]]

The matrix U
[[ 2.   2.  -1.   1. ]
 [ 0.   3.   1.5  1.5]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.   0.   3. ]]
LU - A
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 

#### Порівняння для системи 1 збуреного й незбуреного розв'язку

In [11]:
compare_solutions(start_A_1, start_b_1)

Initial data
[[ 2  2 -1  1]
 [-3  0  3  0]
 [-1  3  3  2]
 [ 1  0  0  4]]
[3, -9, -7, 4]
The new vector b
[3, -9, -7, 4.04]
The solution without perturbation
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

The solution with perturbation 1%
[ 3.98666667 -2.          0.98666667  0.01333333]

Absolute difference:
[0.01333333 0.         0.01333333 0.01333333]


#### Результати для матриці 2 з варіанту №4

In [12]:
start_A_2 = copy.deepcopy(A_2)
start_b_2 = copy.deepcopy(b_2)
print_answers(A_2, b_2) 

The initial matrix A and the vector b
[[  0   3  -1]
 [  9  24   1]
 [ 21  -1 -16]]
[[ 7]
 [20]
 [63]]
The determinant of the matrix A:
1008.0
The solution of the system:
[ 0.  1. -4.]

Answers of libraries
The determinant of the matrix A: 
1008.0000000000002

The solution of the system:
[ 0.  1. -4.]

The error of solution:
[0. 0. 0.]

The matrix can not be factorized in L and U because one of minor is zero

The reverse matrix
[[-0.37996032  0.04861111  0.02678571]
 [ 0.16369048  0.02083333 -0.00892857]
 [-0.50892857  0.0625     -0.02678571]]

The product of the matrix and the reverse matrix
[[ 1.00000000e+00  1.28369537e-16 -5.55111512e-17]
 [ 3.81639165e-17  1.00000000e+00  0.00000000e+00]
 [-4.16333634e-17  9.71445147e-17  1.00000000e+00]]

The conditional number
31.577380952380953


#### Порівняння для системи 2 збуреного й незбуреного розв'язку

In [13]:
compare_solutions(start_A_2, start_b_2)

Initial data
[[  0   3  -1]
 [  9  24   1]
 [ 21  -1 -16]]
[7, 20, 63]
The new vector b
[7, 20, 63.63]
The solution without perturbation
[ 0.  1. -4.]

The solution with perturbation 1%
[ 0.016875  0.994375 -4.016875]

Absolute difference:
[0.016875 0.005625 0.016875]


####  Результати для матриці 3 з варіанту №4

In [15]:
print_answers(A_3, b_3)

The initial matrix A and the vector b
[[ 5  0 -7  0]
 [-1  6  0  1]
 [ 2 -6 -4 -5]
 [-6 -6 15  7]]
[[-123]
 [  60]
 [-108]
 [ 159]]
The determinant of the matrix A is 0

Zero main element occured in the row #3

Answer of libraries
The determinant of the matrix A: 
-2.0783375020982918e-13

The matrix can not be factorized in L and U

The reverse matrix does not exist


#### Результати для матриці й вектора введених з клавіатури

In [23]:
start_matrix = copy.deepcopy(matrix)
start_vector_b = copy.deepcopy(vector_b)
print_answers(matrix, vector_b)

The initial matrix A and the vector b
[[ 2. -7.  1.]
 [ 3. -1.  8.]
 [10. -2.  3.]]
[[15.]
 [12.]
 [18.]]
The determinant of the matrix A:
-466.99999999999994
The solution of the system:
[ 1.21413276 -1.67665953  0.83511777]

Answers of libraries
The determinant of the matrix A: 
-467.00000000000006

The solution of the system:
[ 1.21413276 -1.67665953  0.83511777]

The error of solution:
[0. 0. 0.]

The matrix L
[[1.         0.         0.        ]
 [1.5        1.         0.        ]
 [5.         3.47368421 1.        ]]

The matrix U
[[  2.          -7.           1.        ]
 [  0.           9.5          6.5       ]
 [  0.           0.         -24.57894737]]
LU - A
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.66453526e-15  0.00000000e+00]]

The reverse matrix
[[-0.02783726 -0.04068522  0.11777302]
 [-0.15203426  0.00856531  0.02783726]
 [-0.00856531  0.14132762 -0.04068522]]

The product of the matrix and the

#### Порівняння для системи, введеної з клавіатури, збуреного й незбуреного розв'язку

In [26]:
compare_solutions(start_matrix,start_vector_b)

Initial data
[[ 2. -7.  1.]
 [ 3. -1.  8.]
 [10. -2.  3.]]
[15.15, 12.12, 18.18]
The new vector b
[15.3015, 12.2412, 18.3618]
The solution without perturbation
[ 1.22627409 -1.69342612  0.84346895]

The solution with perturbation 1%
[ 1.23853683 -1.71036039  0.85190364]

Absolute difference:
[0.01226274 0.01693426 0.00843469]


### Висновки:
##### Методом Гауса ми можемо знаходити розв'язки систем лінійних рівнянь, обчислювати детермінант та шукати обернену матрицю. 
#### Проблеми:
##### - може виникати ділення на нуль або маленьке число. Щоб не накопичувалася похибка, ми на кожній ітерації обираємо максимальний елемент в стовпчику й ділимо на нього.
